<a href="https://colab.research.google.com/github/orhysor/qb/blob/main/qb_medilink_V3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title #入力項目 { run: "auto" }

#@markdown ###登録メールアドレス
#@markdown オンラインQBに登録しているメールアドレスを入力してください
my_mail = ''#@param {type:"string"}

#@markdown ###<br>パスワード
#@markdown オンラインQBに登録しているパスワードを入力してください
password = ''#@param {type:"string"}

#@markdown ###<br>Ankiに割り当てるタグ
#@markdown 自由に変更することができます
tag_name = "\u6D88\u5316\u7BA1" #@param {type:"string"}

#@markdown ###<br>保存するフォルダ先
#@markdown Google Driveで設定したフォルダ名と同一にして下さい
file = 'state_exam'#@param {type:"string"}

#@markdown ###<br>出力するCSVファイルの名前
#@markdown 出力するファイル名を入力して下さい
csv_name = 'state_exam_text'#@param {type:"string"}
  #出力するファイル名を''の中に入力して下さい

#@markdown　##<br>問題の解答状況で分類
#@markdown　###◎を含める場合はチェックをつけて下さい
d_circle = False #@param {type:"boolean"}
#@markdown　###○を含める場合はチェックをつけて下さい
circle = False #@param {type:"boolean"}
#@markdown　###△を含める場合はチェックをつけて下さい
tri = False #@param {type:"boolean"}
#@markdown　###×を含める場合はチェックをつけて下さい
cross = True #@param {type:"boolean"}
#@markdown　###未演習を含める場合はチェックをつけて下さい
practice = False #@param {type:"boolean"}

#@markdown ###<br>分野選択
#@markdown 一つ分野を選択して下さい。
search_name = "A \u6D88\u5316\u7BA1" #@param ["A \u6D88\u5316\u7BA1", "B \u809D\u30FB\u80C6\u30FB\u81B5", "C \u5FAA\u74B0\u5668", "D \u4EE3\u8B1D\u30FB\u5185\u5206\u6CCC", "E \u814E\u30FB\u6CCC\u5C3F\u5668", "F \u514D\u75AB\u30FB\u81A0\u539F\u75C5", "G \u8840\u6DB2", "H \u611F\u67D3\u75C7", "I \u547C\u5438\u5668", "J \u795E\u7D4C", "K \u4E2D\u6BD2", "L \u6551\u6025", "M \u9EBB\u9154\u79D1", "N \u533B\u5B66\u7DCF\u8AD6", "O \u5C0F\u5150\u79D1", "P \u5A66\u4EBA\u79D1", "Q \u7523\u79D1", "R \u773C\u79D1", "S \u8033\u9F3B\u54BD\u5589\u79D1", "T \u6574\u5F62\u5916\u79D1", "U \u7CBE\u795E\u79D1", "V \u76AE\u819A\u79D1", "W \u6CCC\u5C3F\u5668\u79D1", "X \u653E\u5C04\u7DDA\u79D1", "Y \u516C\u8846\u885B\u751F", "Z \u5FC5\u4FEE\u554F\u984C"]


In [ ]:
#@title #インストール
#@markdown [shift]+[Enter]を押して起動してください

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

# 日本語フォントインストール
apt install fonts-ipafont-gothic

In [ ]:
#@title #グーグルドライブと連携
#@markdown [shift]+[Enter]を押して起動してください
#@markdown　<br>GogoleDriveとの連携が求められるので、許可してください。

#SeleniumとBeautifulSoupのライブラリをインポート
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from time import sleep

#pandasをインポート
import pandas as pd

#正規表現
import re
import random

#スクロール用
from selenium.webdriver.common.action_chains import ActionChains

#ドライブ設定
PATH_GMOUNT='/content/gdrive'
PATH_MYDRIVE=PATH_GMOUNT+'/My Drive'

#GDriveマウント
from google.colab import drive
drive.mount(PATH_GMOUNT)

In [ ]:
#@title #medilink認証
#@markdown [shift]+[Enter]を押して起動してください
#@markdown <br>エラーが表示される場合はパスワードとメールアドレスが間違っていないか確認して下さい。

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
user_agent =[
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko)",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko)",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.3 Safari/605.1.15",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36"
]

UA = user_agent[random.randrange(0, len(user_agent), 1)]
options.add_argument('--user-agent=' + UA)
driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(10)

driver.get("https://qb.medilink-study.com/")

#ポップアップ処理
dialog = driver.find_element(by=By.CLASS_NAME, value="news-dialog__button")
if dialog is not None:
    dialog.click()

#ログイン画面へ
LoginButton = driver.find_element(by=By.CLASS_NAME, value="login__button--login")
LoginButton.click()

#　ログイン項目を入力
mail_form = driver.find_element(by=By.NAME, value='username')
mail_form.send_keys(my_mail)

password_form = driver.find_element(by=By.NAME, value='password')
password_form.send_keys(password)

#ログイン
login_mypage = driver.find_element(by=By.ID, value="js-login-button")
login_mypage.click()

wait = WebDriverWait(driver, 15)
def WaitClass(class_name):
    driver.set_page_load_timeout(15)
    driver.set_script_timeout(15)
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, class_name)))

WaitClass("contents-qb__tab")

if driver.current_url == "https://qb.medilink-study.com/MyPage":
    print("ログインに成功しました")
else:
    print("ログインに失敗しました。メールアドレス、パスワードがあっているか確認して再実行して下さい。")

In [ ]:
#@title #テキストと画像の取得
#@markdown [shift]+[Enter]を押して起動してください
#@markdown　<br>ランタイムの接続が切断されるまで、2回目以降はこのセルの実行だけで問題をコピーできます。

data = []

#Mypageへ
driver.get('https://qb.medilink-study.com/MyPage')

#画面リセット
WaitClass("contents-qb__tab")
tab_menu = driver.find_elements(by=By.CLASS_NAME,value = "contents-qb__tab")
tab_menu[1].click()
tab_menu[0].click()

#分野名抽出
#chapters = driver.find_elements_by_class_name('chapterLeft--title')
chapters = driver.find_elements(by=By.CLASS_NAME,value = "chapterLeft--title")
WaitClass('chapterLeft')
a = []
bbb = []
for chapter in chapters:
    cha = re.sub('\W\d*\W','',chapter.text)
    bbb.append(cha)

#Mypageから各分野に移動
#chapterlefts = driver.find_elements_by_class_name('chapterLeft')
chapterlefts = driver.find_elements(by=By.CLASS_NAME,value = 'chapterLeft')
chapterlefts[bbb.index(search_name)].click()

#Questionsに移動
#chapterleft = driver.find_element_by_class_name('chapterLeft')
chapterleft = driver.find_element(by=By.CLASS_NAME,value = 'chapterLeft')
WaitClass('chapterLeft')
chapterleft.click()


#条件の変更
modal_terms = [d_circle, circle, tri, cross, practice]
for i, modal in enumerate(modal_terms):
    WaitClass("refine-result__item")
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "refine-result__item")))
    #elements = driver.find_elements_by_class_name("refine-result__item")
    elements = driver.find_elements(by=By.CLASS_NAME,value = "refine-result__item")
    elem = elements[i]
    color = elem.value_of_css_property("background-color")
    if modal:
        if color == 'rgba(0, 0, 0, 0)':
            driver.execute_script('arguments[0].click();', elem)
    else:
        if color != 'rgba(0, 0, 0, 0)':
            driver.execute_script('arguments[0].click();', elem)

#numbers = driver.find_elements_by_class_name("table-body__number")
numbers = driver.find_elements(by=By.CLASS_NAME,value = "table-body__number")
numbers[0].click()
data = []

while True:

    driver.set_page_load_timeout(15)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    #ボタン待機
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    btns = soup.find_all(attrs={'class': 'btn'})

    i = 1

    while len(btns) <=1 and i <= 3:
        sleep(3)
        html = driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
        btns = soup.find_all(attrs={'class': 'btn'})
        i += 1

    if len(btns) <=1 and i == 4:
        print(f"{driver.current_url}の取得に失敗しました")
        next_q = driver.find_element(by=By.CLASS_NAME,value = "custom-icon-arrow_right")
        next_q.click()
        continue

    i = 1
    btns = driver.find_elements(by=By.CLASS_NAME,value = "btn")
    btn = btns[1]

    #ロードされるまで待機
    while btn.text != '解答を確認する' and i <= 3:
        sleep(2)
        btns = driver.find_elements(by=By.CLASS_NAME,value = "btn")
        btn = btns[1]
        i += 1


    if btn.text != '解答を確認する' and i == 4:
        print(f"{driver.current_url}の取得に失敗しました")
        next_q = driver.find_element(by=By.CLASS_NAME,value = "custom-icon-arrow_right")
        next_q.click()
        continue

    #ボタンの定義
    btns = driver.find_elements(by=By.CLASS_NAME,value = "btn")
    btn = btns[1]

    #現在のurl取得
    url = driver.current_url
    question_number = re.findall("\d+.+", url)[0]

    try:
        #読み込み待機時間
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "question-main")))

    except TimeoutException as e:
        print(f'問題番号{question_number}が見つかりませんでした。')
        next_q = driver.find_element(by=By.CLASS_NAME,value = "custom-icon-arrow_right")
        next_q.click()
        continue

    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

    answer_item = soup.find(attrs={'class': 'answer-item'})

    #消去関数の定義
    pattern_data = 'data-v-\w*=""'
    pattern_doc = 'doc_id="\w*"'
    pattern_out = '<!-- -->'

    def PatternDelete(content):
        content = re.sub(pattern_data, '', content)
        content = re.sub(pattern_doc, '', content)
        content = re.sub(pattern_out, '', content)
        content = content.replace("\u3000"," ")
        return content

    def GetClass(class_name):
        class_name = soup.find(attrs={'class': class_name})
        class_name = str(class_name)
        class_name = PatternDelete(class_name)
        return class_name

    #解答をみるを消去する
    if answer_item is not None:
        answer_item.decompose()

    #問題文整形
    question_container = GetClass('question-main')

    #解答を表示
    btn.click()

    try:
        #読み込み待機時間
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "descContent--title")))

    except TimeoutException as e:
        print(f"{driver.current_url}の取得に失敗しました")
        next_q = driver.find_element(by=By.CLASS_NAME,value = "custom-icon-arrow_right")
        next_q.click()
        continue

    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

    #解答の抽出
    numbers = soup.find_all(attrs={'class': 'resultContent--number'})
    anss = soup.find_all(attrs={'class': 'resultContent--currentCorrectAnswer'})
    answers = ''
    for number, ans in zip(numbers,anss):
        answers += f'<div class = answer_wrapper ><div class = answer_number >{number.text}</div><div class = correct_answer>正解： {ans.text}</div></div><br>'

    #解説の抽出
    desc = GetClass('desc--main')

    #基本事項の抽出
    basic = GetClass('basic--main')

    #画像の抽出と置換
    imgs = soup.find_all(attrs={'class': 'images'})
    if imgs != []:

        bs_imgs = soup.find_all('img')
        for i, img in enumerate(bs_imgs):
            img = str(img)
            re_img = f'<img src="{question_number}_{i+1}.png" />'
            question_container = question_container.replace(img,re_img)
            desc = desc.replace(img,re_img)
            basic = basic.replace(img,re_img)

        sleep(5)
        imgs = driver.find_elements(by = By.TAG_NAME, value = "img")
        for i,img in enumerate(imgs):
            with open(f"{PATH_MYDRIVE}/{file}/{question_number}_{i+1}.png", 'wb') as f:
                try:
                    f.write(img.screenshot_as_png)
                    sleep(1)

                except:
                    print(f"{driver.current_url}の取得に失敗しました")
                    next_q = driver.find_element(by=By.CLASS_NAME,value = "custom-icon-arrow_right")
                    next_q.click()
                    continue
    datum = {}
    datum['問題番号'] = question_number
    datum['問題'] = question_container
    datum['解答'] = answers
    datum['解説リンク'] = url
    datum['解説'] = desc
    datum['基本事項'] = basic
    datum['タグ'] = tag_name

    data.append(datum)

    print(f'{question_number}の取得が完了しました。')

    sleep(2)

    next_q = driver.find_element(by=By.CLASS_NAME,value = "toNextWrapper--btn")
    next_q.click()

    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

    if soup.find_all(attrs={'class': 'popup__inner'}) != []:
        break

df = pd.DataFrame(data)

#Google Driveにデータを出力する
df.to_csv(f'{PATH_MYDRIVE}/{file}/{csv_name}.csv', index=False, header = False)
print(f'GoogleDriveに{csv_name}.csvが出力されました')
print('https://drive.google.com/drive/u/0/my-drive')